# Loan Repayment 
## Exploratory Data Analysis (EDA)

In [1]:
import os
# Change directory to this file's directory
thisPath = globals()['_dh'][-1]
os.chdir(thisPath)
print("This file's directory:", os.getcwd())
# Change current path to parent of this file's directory
# to access all modules from parent
os.chdir('..')
print("Parent directory:", os.getcwd())

This file's directory: /home/mattkhoo/Git-Loan-Repayment-EDA-Predictive/notebooks
Parent directory: /home/mattkhoo/Git-Loan-Repayment-EDA-Predictive


In [3]:
from main.data_loader import DataLoader
import main.viz_utils as viz
%matplotlib inline